In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys
import os
sys.path.append(os.path.abspath('./odibi_de_v2'))
os.environ["PYTHONDONTWRITEBYTECODE"] = "1"
from odibi_de_v2.connector import AzureBlobConnection
from odibi_de_v2.ingestion import ReaderProvider
from odibi_de_v2.core import DataType, Framework
from pyspark.sql import SparkSession
from odibi_de_v2.storage import SaverProvider


In [0]:
azure_spark_connector = AzureBlobConnection(
    dbutils.secrets.get("", ""),
    dbutils.secrets.get("", ""),
    Framework.SPARK)

provider = ReaderProvider(azure_spark_connector)

df = provider.read(
    spark=spark,
    data_type=DataType.AVRO,
    container="",
    path_prefix="",
    object_name="",
    is_stream=False)

schema_file_path = azure_spark_connector.get_file_path(
    "",
    "saver_test/schema",
    ""
)
streaming_df = provider.read(
    data_type=DataType.AVRO,
    container="",
    path_prefix="",
    object_name="",
    spark=spark,
    is_stream=True,
    options={"cloudFiles.schemaLocation": schema_file_path})


In [0]:
saver_provider = SaverProvider(azure_spark_connector)
saver_provider.save(
    df=df,
    data_type=DataType.CSV,
    container="",
    path_prefix="",
    object_name="test_save_spark_csv",
    spark=spark,
    mode="overwrite"
)


# Streaming 


checkpoint_file_path = azure_spark_connector.get_file_path(
    "",
    "saver_test/checkpoint",
    ""
)

saver_provider.save(
    df=streaming_df,
    data_type=DataType.DELTA,
    container="",
    path_prefix="saver_test",
    object_name="test_save_spark_delta",
    spark=spark,
    is_stream=True,
    options={"checkpointLocation": checkpoint_file_path},
    trigger={"availableNow": True}
)